In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading KB Tuples**

In [ ]:
%cd /content 

/content


In [ ]:
!wget https://ai2-public-datasets.s3.amazonaws.com/tuple-kb/aristo-tuple-kb-v4-mar2017.zip

--2021-09-28 20:33:50--  https://ai2-public-datasets.s3.amazonaws.com/tuple-kb/aristo-tuple-kb-v4-mar2017.zip
Resolving ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)... 52.218.252.107
Connecting to ai2-public-datasets.s3.amazonaws.com (ai2-public-datasets.s3.amazonaws.com)|52.218.252.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23190065 (22M) [binary/octet-stream]
Saving to: ‘aristo-tuple-kb-v4-mar2017.zip’

aristo-tuple-kb-v4- 100%[===================>]  22.12M  20.5MB/s    in 1.1s    

2021-09-28 20:33:51 (20.5 MB/s) - ‘aristo-tuple-kb-v4-mar2017.zip’ saved [23190065/23190065]



In [ ]:
!unzip /content/aristo-tuple-kb-v4-mar2017.zip

Archive:  /content/aristo-tuple-kb-v4-mar2017.zip
   creating: aristo-tuple-kb-v4-mar2017/
  inflating: aristo-tuple-kb-v4-mar2017/COMBINED-KB.tsv  
   creating: __MACOSX/
   creating: __MACOSX/aristo-tuple-kb-v4-mar2017/
  inflating: __MACOSX/aristo-tuple-kb-v4-mar2017/._COMBINED-KB.tsv  
  inflating: aristo-tuple-kb-v4-mar2017/Creative Commons - Attribution-ShareAlike 4.0 International - CC BY-SA 4.pdf  
  inflating: __MACOSX/aristo-tuple-kb-v4-mar2017/._Creative Commons - Attribution-ShareAlike 4.0 International - CC BY-SA 4.pdf  
  inflating: aristo-tuple-kb-v4-mar2017/Domain Vocabulary - mar2016.tsv  
  inflating: __MACOSX/aristo-tuple-kb-v4-mar2017/._Domain Vocabulary - mar2016.tsv  
  inflating: aristo-tuple-kb-v4-mar2017/entities.tsv  
  inflating: __MACOSX/aristo-tuple-kb-v4-mar2017/._entities.tsv  
   creating: aristo-tuple-kb-v4-mar2017/kb-components/
  inflating: aristo-tuple-kb-v4-mar2017/kb-components/animal-negatives.tsv  
   creating: __MACOSX/aristo-tuple-kb-v4-mar2017

# **Libkge Preprocessing**

In [ ]:
kb_file = '/content/aristo-tuple-kb-v4-mar2017/COMBINED-KB.tsv'
import pandas as pd

df = pd.read_csv(kb_file, sep='\t')
df = df[['Arg1', 'Pred', 'Arg2']]
df.head()

,Arg1,Pred,Arg2
0,aardvark,has-part,brain
1,aardvark,has-part,breast
2,aardvark,has-part,cell membrane
3,aardvark,has-part,cell
4,aardvark,has-part,chest cavity


In [ ]:
from sklearn.model_selection import GroupKFold
fold = 0
gkf  = GroupKFold(n_splits = 15)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df, groups = df.Arg1.tolist())):
    df.loc[val_idx, 'fold'] = fold
df.head()

,Arg1,Pred,Arg2,fold
0,aardvark,has-part,brain,8
1,aardvark,has-part,breast,8
2,aardvark,has-part,cell membrane,8
3,aardvark,has-part,cell,8
4,aardvark,has-part,chest cavity,8


In [ ]:
fold = 0
kb_train = ""
kb_test = ""
train_count = 0
test_count = 0
from tqdm import tqdm
for i in tqdm(range(len(df))):
    row = df.loc[i]

    Arg1 = str(row.Arg1).replace(" ", "/")
    Pred = str(row.Pred).replace(" ", "/")
    Arg2 = str(row.Arg2).replace(" ", "/")

    if row.fold == fold:
      kb_test = kb_test + Arg1 + "\t" + Pred + "\t" + Arg2 + '\n'
      train_count += 1
    else:
      kb_train = kb_train + Arg1 + "\t" + Pred + "\t" + Arg2 + '\n'
      test_count += 1
print(train_count, test_count)

100%|██████████| 282594/282594 [04:56<00:00, 954.66it/s]

18840 263754


In [ ]:
import os, shutil
os.mkdir('/content/kge/data/aristo/')

In [ ]:
#Train.txt and Test.txt
with open('/content/drive/MyDrive/Libkge/data/aristo/train.txt', 'w') as f:
  f.writelines(kb_train)

with open('/content/drive/MyDrive/Libkge/data/aristo/test.txt', 'w') as f:
  f.writelines(kb_test)

with open('/content/drive/MyDrive/Libkge/data/aristo/valid.txt', 'w') as f:
  f.writelines(kb_test)

In [ ]:
#Entities.txt
def add_newline(p):
  return p + '\n'

entities_df = pd.read_csv('/content/aristo-tuple-kb-v4-mar2017/entities.tsv', sep='\t')
entities = entities_df['Word'].apply(lambda x: str(x).replace(" ", "/")).tolist()
entities = list(map(add_newline, entities))
with open('/content/kge/data/aristo/entities.txt', 'w') as f:
  f.writelines(entities)
print(entities_df['Word'][0])

In [ ]:
#Relations.txt
relations_df = pd.read_csv('/content/aristo-tuple-kb-v4-mar2017/relations.tsv', sep='\t')
relations = relations_df['Relation'].apply(lambda x: str(x).replace(" ", "/")).tolist()
relations = list(map(add_newline, relations))
with open('/content/kge/data/aristo/relations.txt', 'w') as f:
  f.writelines(relations)
print(relations_df['Relation'][0])
relations_df.head()

In [ ]:
shutil.copy('/content/kge/examples/toy-complex-search-grid.yaml', '/content/kge/examples/aristo.yaml')

In [ ]:
shutil.make_archive('/content/kge/data/aristo', 'zip', '/content/kge/data/aristo')

In [ ]:
from google.colab import files
files.download("/content/kge/data/aristo.zip")

# **Embeddings using LibKge**

In [ ]:
!git clone https://github.com/uma-pi1/kge.git

Cloning into 'kge'...
remote: Enumerating objects: 7561, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 7561 (delta 56), reused 57 (delta 29), pack-reused 7465
Receiving objects: 100% (7561/7561), 1.82 MiB | 578.00 KiB/s, done.
Resolving deltas: 100% (5709/5709), done.


In [ ]:
ls

kge/  sample_data/


In [ ]:
!cp -r /content/drive/MyDrive/kge /content/

cp: cannot stat '/content/drive/MyDrive/kge': No such file or directory


In [ ]:
%cd /content/drive/MyDrive/kge

/content/drive/MyDrive/kge


In [ ]:
!pip install -e . 

In [ ]:
# %%shell
# cd /content/kge/data
# sh download_all.sh
# cd ..

In [ ]:
# !tar xvf toy.tar.gz

In [ ]:
%cd /content/drive/MyDrive/kge/data

/content/drive/MyDrive/kge/data


In [ ]:
# !rm -rf /content/kge/data/aristo

In [ ]:
# !cp -r /content/drive/MyDrive/Libkge/data/aristo /content/kge/data/

In [ ]:
# !cp -r /content/EmbedKGQA/data/aristo /content/kge/data/

In [ ]:
!cp /content/kge/examples/toy-complex-train.yaml /content/kge/examples/SciQ.yaml

In [ ]:
%cd /content/kge/data

/content/kge/data


In [ ]:
!python preprocess/preprocess_default.py SciQ

In [ ]:
# !cp -r /content/kge/data/aristo /content/EmbedKGQA/data/

In [ ]:
# !cp /content/kge/examples/aristo.yaml /content/drive/MyDrive/Libkge/data/

In [ ]:
!cp -r /content/drive/MyDrive/kge/data/SciQ /content/kge/data

In [ ]:
!kge start /content/drive/MyDrive/kge/examples/SciQ.yaml --job.device cuda:0

In [ ]:
!cp -r /content/kge/local/20211014-142756-SciQ /content/drive/MyDrive/Libkge/experiments

cp: cannot stat '/content/kge/local/20211014-142756-SciQ': No such file or directory


In [ ]:
!cp /content/drive/MyDrive/kge/examples/SciQ.yaml /content/kge/examples

In [ ]:
!kge valid /content/kge/examples/aristo.yaml 

In [ ]:
# cd /content/drive/MyDrive/kge/

In [ ]:
# ls

In [ ]:
# cd /content/kge

In [ ]:
# !kge dump checkpoint /content/kge/examples/checkpoint_best.pt

In [ ]:
%cd /content/drive/MyDrive/kge

/content/drive/MyDrive/kge


In [ ]:
ls

CHANGELOG.md     docs/       kge/              local/     setup.py
CONTRIBUTING.md  EmbedKGQA/  libkge.egg-info/  README.md  tests/
data/            examples/   LICENSE           setup.cfg


In [ ]:
import torch
from kge.model import KgeModel
from kge.util.io import load_checkpoint

# download link for this checkpoint given under results above
checkpoint = load_checkpoint('/content/drive/MyDrive/kge/examples/checkpoint_best.pt')
model = KgeModel.create_from(checkpoint)

s = torch.Tensor([0,]).long()             # subject indexes
p = torch.Tensor([0,]).long()             # relation indexes
scores = model.score_sp(s, p)                # scores of all objects for (s,p,?)
o = torch.argmax(scores, dim=-1)             # index of highest-scoring objects

print(o)
print(model.dataset.entity_strings(s))       # convert indexes to mentions
print(model.dataset.relation_strings(p))
print(model.dataset.entity_strings(o))

# Output (slightly revised for readability):
#
# tensor([8399, 8855])
# ['Dominican Republic'        'Mighty Morphin Power Rangers']
# ['has form of government'    'is tv show with actor']
# ['Republic'                  'Johnny Yong Bosch']


Loading configuration of dataset SciQ from /content/drive/MyDrive/kge/data/SciQ ...
tensor([1])
Loaded 15 keys from map entity_strings
['sperm']
Loaded 9 keys from map relation_strings
['forming']
['diploid']


In [ ]:
# !cp -r /content/kge/local/experiments/20210928-234059-aristo /content/drive/MyDrive/Libkge/experiments/

In [ ]:
# !kge resume /content/kge/examples/aristo.yaml --job.device cuda:0

# **KGQA**

In [ ]:
# !pip install torch==1.6.0 torchvision==0.7.0

In [ ]:
!pip install transformers
!pip install Path
!pip install sentencepiece

     |████████████████████████████████| 2.9 MB 2.7 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 3.3 MB 35.2 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


     |████████████████████████████████| 1.2 MB 2.0 MB/s 


In [ ]:
%cd /content

/content


In [ ]:
!git clone https://github.com/malllabiisc/EmbedKGQA.git

Cloning into 'EmbedKGQA'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 254 (delta 67), reused 145 (delta 43), pack-reused 53
Receiving objects: 100% (254/254), 67.86 MiB | 26.41 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [ ]:
!cp -r /content/drive/MyDrive/EmbedKGQA/data /content/EmbedKGQA

In [ ]:
!cp -r /content/drive/MyDrive/EmbedKGQA/pretrained_models /content/EmbedKGQA 

In [ ]:
import os
os.mkdir('/content/EmbedKGQA/pretrained_models')
os.mkdir('/content/EmbedKGQA/pretrained_models/embeddings')
os.mkdir('/content/EmbedKGQA/pretrained_models/embeddings/SciQ')

os.mkdir('/content/EmbedKGQA/data')
os.mkdir('/content/EmbedKGQA/data/QA_data')


In [ ]:
!cp -r /content/drive/MyDrive/kge/data/SciQ /content/EmbedKGQA/data/

In [ ]:
!cp /content/drive/MyDrive/kge/data/SciQ/entity_ids.del /content/EmbedKGQA/pretrained_models/embeddings/SciQ/

In [ ]:
# !cp -r /content/drive/MyDrive/Libkge/data/aristo /content/EmbedKGQA/data

In [ ]:
!cp /content/drive/MyDrive/kge/examples/checkpoint_best.pt /content/EmbedKGQA/pretrained_models/embeddings/SciQ

In [ ]:
# !cp /content/EmbedKGQA/data/aristo/entity_ids.del /content/EmbedKGQA/pretrained_models/embeddings/aristo/

In [ ]:
# !cp -r /content/drive/MyDrive/kge /content/

In [ ]:
%cd /content/EmbedKGQA

/content/EmbedKGQA


In [ ]:
# !gdown --id 1uWaavrpKKllVSQ73TTuLWPc4aqVvrkpx && unzip data.zip

In [ ]:
# !gdown --id 1Ly_3RR1CsYDafdvdfTG35NPIG-FLH-tz && unzip pretrained_models.zip

In [ ]:
# !cp -r /content/EmbedKGQA/EmbedKGQA/data /content/EmbedKGQA

In [ ]:
# !cp -r /content/EmbedKGQA/EmbedKGQA/pretrained_models /content/EmbedKGQA

In [ ]:
# import shutil
# shutil.copytree('/content/kge/data/aristo', '/content/EmbedKGQA/data/aristo')

In [ ]:
# !cp -r /content/kge/local/experiments/20210928-234059-aristo/ /content/EmbedKGQA/pretrained_models/embeddings/aristo/

In [ ]:
# !cp /content/kge/local/experiments/20210928-234059-aristo/checkpoint_best.pt /content/EmbedKGQA/pretrained_models/embeddings/aristo/

In [ ]:
# !cp /content/EmbedKGQA/data/aristo/entity_ids.del /content/EmbedKGQA/pretrained_models/embeddings/aristo/

In [ ]:
# !rm -rf /content/EmbedKGQA

In [ ]:
!cp -r /content/drive/MyDrive/EmbedKGQA /content

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [ ]:
%cd /content/EmbedKGQA/KGQA/RoBERTa 

/content/EmbedKGQA/KGQA/RoBERTa


In [ ]:
# !cp -r /content/EmbedKGQA /content/drive/MyDrive

In [ ]:
import os
os.mkdir('/content/EmbedKGQA/output/')

In [ ]:
!cp /content/drive/MyDrive/EmbedKGQA/KGQA/RoBERTa/main.py /content/EmbedKGQA/KGQA/RoBERTa

In [ ]:
!cp /content/drive/MyDrive/EmbedKGQA/data/QA_data/aristoQA_train.txt /content/EmbedKGQA/data/QA_data/

In [ ]:
!python3 main.py --mode train --relation_dim 50 --do_batch_norm 1 \
--gpu 0 --freeze 1 --batch_size 16 --validate_every 10 --hops aristo --lr 0.00002 --entdrop 0.0 --reldrop 0.0 --scoredrop 0.0 \
--decay 1.0 --model ComplEx --patience 20 --ls 0.05 --l3_reg 0.001 --nb_epochs 10 --outfile sciq

Loading entities and relations
Loading kg embeddings from ../../pretrained_models/embeddings/checkpoint_best.pt
Loading configuration of dataset SciQ from /content/drive/MyDrive/kge/data/SciQ ...
Loaded entities and relations
data_line
['What is the stored food in a seed called? [food]', 'endosperm']
question
['What is the stored food in a seed called? ', 'food]']
data
[['food', 'What is the stored food in a seed called? NE', ['endosperm']]]
Train file processed, making dataloader
Creating model...
Downloading: 100% 478M/478M [00:11<00:00, 45.2MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model fro

In [ ]:
!pwd

/content/EmbedKGQA/KGQA/RoBERTa


In [ ]:
!python3 main.py --mode eval --load_from aristo_reduced --relation_dim 50 --do_batch_norm 1 \
--gpu 0 --freeze 1 --batch_size 16 --validate_every 10 --hops aristo --lr 0.00002 --entdrop 0.0 --reldrop 0.0 --scoredrop 0.0 \
--decay 1.0 --model ComplEx --patience 20 --ls 0.05 --l3_reg 0.001 --nb_epochs 20 --outfile aristo_reduced

Loading entities and relations
Loading kg embeddings from ../../pretrained_models/embeddings/aristo/checkpoint_best.pt
Loading configuration of dataset aristo from /content/drive/MyDrive/kge/data/aristo ...
Loaded entities and relations
Evaluation file processed, making dataloader
Creating model...
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model

In [ ]:
# 1) Score 0.03832752613240418
!cp -r /content/EmbedKGQA /content/drive/MyDrive/

# **Run Inference**

In [ ]:
from tqdm import tqdm

def load_questions_answers(question_path, ans_choice_path):

  ans_choices = []
  with open(ans_choice_path) as f:
    choices = f.readlines()
    # print(choices)
    for choice in choices:
      choice = choice.strip("\n").split('\t')
      ans_choices.append([choice[0], choice[1], choice[2], choice[3]])

  questions = []
  with open(question_path) as f:
    qs = f.readlines()
    # print(questions)
    for ques in qs:
      ques = ques.strip("\n")
      questions.append(ques)
      

  return questions, ans_choices

question_path = '/content/EmbedKGQA/data/QA_data/questions.txt'
ans_choice_path = '/content/EmbedKGQA/data/QA_data/QA_choices.txt'
questions, ans_choices = load_questions_answers(question_path, ans_choice_path)
print(len(questions), len(ans_choices))

287 287


In [ ]:
import shutil
shutil.copytree("/content/EmbedKGQA", '/content/drive/MyDrive/EmbedKGQA_final')

'/content/drive/MyDrive/EmbedKGQA_final'

In [ ]:
%cd /content/EmbedKGQA/KGQA/RoBERTa

/content/EmbedKGQA/KGQA/RoBERTa


In [ ]:
!pwd

/content/EmbedKGQA/KGQA/RoBERTa


In [ ]:
!cp /content/drive/MyDrive/EmbedKGQA/output/aristo_reduced.pt /content/EmbedKGQA/output/

In [ ]:
import torch
device = torch.device(0)

In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
from dataloader import DatasetMetaQA, DataLoaderMetaQA
from model import RelationExtractor
import networkx as nx
from kge.model import KgeModel
from kge.util.io import load_checkpoint

def getEntityEmbeddings(kge_model):
    e = {}
    entity_dict = '../../pretrained_models/embeddings/aristo/entity_ids.del'
    embedder = kge_model._entity_embedder
    f = open(entity_dict, 'r')
    for line in f:
        line = line[:-1].split('\t')
        ent_id = int(line[0])
        ent_name = line[1]
        e[ent_name] = embedder._embeddings(torch.LongTensor([ent_id]))[0]
    f.close()
    return e

def prepare_embeddings(embedding_dict):
    entity2idx = {}
    idx2entity = {}
    i = 0
    embedding_matrix = []
    for key, entity in embedding_dict.items():
        entity2idx[key] = i
        idx2entity[i] = key
        i += 1
        embedding_matrix.append(entity)
    return entity2idx, idx2entity, embedding_matrix


embedding_dim = 256
relation_dim = 50
model_name = 'ComplEx'
do_batch_norm = 1

def load_models(load_from):
    
    #Load KG Embeddings
    checkpoint_file = '../../pretrained_models/embeddings/aristo/checkpoint_best.pt'
    print('Loading kg embeddings from', checkpoint_file)
    kge_checkpoint = load_checkpoint(checkpoint_file)
    kge_model = KgeModel.create_from(kge_checkpoint)
    kge_model.eval()
    e = getEntityEmbeddings(kge_model)

    print('Loaded entities and relations')

    entity2idx, idx2entity, embedding_matrix = prepare_embeddings(e)


    #Load RE Model
    model = RelationExtractor(embedding_dim=embedding_dim, num_entities = len(idx2entity), relation_dim=relation_dim, 
                              pretrained_embeddings=embedding_matrix, device=device, 
                              model = model_name, do_batch_norm=do_batch_norm)
      
    fname = "/content/EmbedKGQA/output/" + load_from + ".pt"
    print('Loading from %s' % fname)
    model.load_state_dict(torch.load(fname, map_location=torch.device('cpu')))
    print('Loaded successfully!')
    model.to(device)

    return model, e, entity2idx, idx2entity, embedding_matrix

def get_prediction(model, question, ans_choices, e, entity2idx, idx2entity, embedding_matrix):

  #Getting head and question
  # print(question)
  question = question.split('[')
  question_1 = question[0]
  question_2 = question[1].split(']')
  head = question_2[0].strip()
  question_2 = question_2[1]
  question = question_1+'NE'+question_2
  # print(question, head)
  head = entity2idx[head.strip()]
  head = torch.tensor(head, dtype=torch.long)
  head = head.to(device)
  data = [head, question]
  # Get_prediction
  dataloader = DatasetMetaQA(data, e, entity2idx)
  
  question_tokenized, attention_mask = dataloader.tokenize_question(question)
  question_tokenized = question_tokenized.to(device)
  attention_mask = attention_mask.to(device)

  # print(head)
  pred = model.get_score_ranked(head=head, question_tokenized=question_tokenized, attention_mask=attention_mask)[0]
  pred = pred.view(1, 44948)
  pred = torch.nn.functional.normalize(pred, p=2.0, dim=1, eps=1e-12, out=None)
  # print('*****pred_embedding******')
  # print(pred.size())

  ans_embeddings = []
  for choice in ans_choices:
    ans_tokenized, ans_attention_mask = dataloader.tokenize_question(choice)
    ans_tokenized = ans_tokenized.to(device)
    ans_attention_mask = ans_attention_mask.to(device)
    ans_embedding = model.getQuestionEmbedding(ans_tokenized.unsqueeze(0), ans_attention_mask.unsqueeze(0))
    
    hidden2rel_ans = nn.Linear(768, 44948).to(device)
    ans_embedding = hidden2rel_ans(ans_embedding)
    # ans_embedding = torch.nn.functional.normalize(ans_embedding, p=2.0, dim=1, eps=1e-12, out=None)
    ans_embeddings.append(ans_embedding)

  #Select ans which is closest to predicted value
  minimum_dist = float('inf')
  ans_index = -1
  for i, ans in enumerate(ans_embeddings):
    
    output = torch.cdist(ans,pred)
    # print(output)
    if minimum_dist > output:
      minimum_dist = output
      ans_index = i

  return ans_choices[ans_index]

In [ ]:
model, e, entity2idx, idx2entity, embedding_matrix = load_models('aristo_reduced')
total = len(questions)
correct = 0
count = 0
for ques, ans in tqdm(zip(questions, ans_choices)):

  pred = get_prediction(model, ques, ans, e, entity2idx, idx2entity, embedding_matrix)
  if pred == ans[-1]:
    correct += 1
  count += 1
  if (count == 10):
    count = 0
    print(correct)
print(correct)
accuracy = correct/total * 100

print(f'accuracy = {accuracy}%')


In [ ]:
model, e, entity2idx, idx2entity, embedding_matrix = load_models('aristo_reduced')

Loading kg embeddings from ../../pretrained_models/embeddings/aristo/checkpoint_best.pt
Loading configuration of dataset aristo from /content/drive/MyDrive/kge/data/aristo ...
Loaded entities and relations


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model is ComplEx
Frozen: True
torch.Size([44948, 100])
Loading from /content/EmbedKGQA/output/aristo_reduced.pt
Loaded successfully!


In [ ]:
# model, e, entity2idx, idx2entity, embedding_matrix = load_models('aristo_final')
total = len(questions)
correct = 0
print("***question***")
print(questions[0])
# questions[2] = questions[2].replace("presence", "vertebrate")
correct_ans = get_prediction(model, questions[0], ans_choices[0], e, entity2idx, idx2entity, embedding_matrix)



print("***correct_ans***")
print(correct_ans)
print(ans_choices[0])

***question***
Vertebrata are characterized by the presence of what? [presence]
tensor([[97.4000]], device='cuda:0', grad_fn=<CdistBackward>)
tensor([[97.1697]], device='cuda:0', grad_fn=<CdistBackward>)
tensor([[96.8201]], device='cuda:0', grad_fn=<CdistBackward>)
tensor([[97.3971]], device='cuda:0', grad_fn=<CdistBackward>)
***correct_ans***
Thumbs
['Bones', 'Muscles', 'Thumbs', 'backbone']


In [ ]:
# model, e, entity2idx, idx2entity, embedding_matrix = load_models('aristo_final')
total = len(questions)
correct = 0
question = 'Testing what usually requires making observations or performing experiments?'
ans_choices = ['']
# questions[2] = questions[2].replace("presence", "vertebrate")
correct_ans = get_prediction(model, question, ans_choices, e, entity2idx, idx2entity, embedding_matrix)

print("***question***")
print(question)

print("***correct_ans***")
print(correct_ans)
print(ans_choice)